In [7]:
import os
import tensorflow as tf 

from sklearn import metrics
from sklearn.metrics import classification_report

### RUN THIS CELL IF YOU ARE RUNNING THIS NOTEBOOK LOCALLY

In [8]:
if os.getcwd().split('/')[-1] == "evaluation":
    %cd ../..

from scripts.physical_transformation import get_direct_noisy_image
from scripts.model import VGG

## Define noise level for experiment

In [31]:
NOISE_LEVEL = 0.05
TEST_NOISE_LEVELS = [0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5]

## Load Data

In [34]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator()
val_datagen = tf.keras.preprocessing.image.ImageDataGenerator()

# Generators
train_generator = train_datagen.flow_from_directory(
    'data/RSNA_BME548-png/test_noisy_0_05',
    target_size=(224, 224),
    batch_size=4,
    class_mode='binary'
)

validation_generator = val_datagen.flow_from_directory(
    'data/RSNA_BME548-png/test_noisy_0_15',
    target_size=(224, 224),
    batch_size=4,
    class_mode='binary'
)

Found 800 images belonging to 2 classes.
Found 800 images belonging to 2 classes.


In [35]:
def train_model():

  optimizer = tf.keras.optimizers.Adam(
    learning_rate=0.0005,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-07,
    amsgrad=False
  )

  model = VGG()
  model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
  model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator)

  return model

In [36]:
def get_predictions(model, data_gen):
    # Load a batch of images from the dataset
    images, labels = next(iter(data_gen))

    # Get model predictions
    preds = model.predict(images)

    preds_binary = (preds > 0.5).astype(int)

    return preds_binary

In [37]:
def get_classification_report(y_test, y_pred):
  accuracy = metrics.accuracy_score(y_test,y_pred)

  print("Accuracy of model=",accuracy)

  return classification_report(y_test, y_pred)

In [38]:
# Train the model
model = train_model()

Epoch 1/10


/Users/jc/.venvs/bme548/lib/python3.11/site-packages/keras/src/layers/layer.py:360: UserWarning: `build()` was called on layer 'vgg_3', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(
/Users/jc/.venvs/bme548/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


200/200 ━━━━━━━━━━━━━━━━━━━━ 382s 2s/step - accuracy: 0.5517 - loss: 1.6869 - val_accuracy: 0.6862 - val_loss: 0.6595
Epoch 2/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 414s 2s/step - accuracy: 0.6568 - loss: 0.7805 - val_accuracy: 0.6212 - val_loss: 0.7112
Epoch 3/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 441s 2s/step - accuracy: 0.6354 - loss: 0.7387 - val_accuracy: 0.5487 - val_loss: 0.8308
Epoch 4/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.7073 - loss: 0.6894

KeyboardInterrupt: 

In [27]:
def get_test_data():
    test_gens = {}
    for nl in TEST_NOISE_LEVELS:
        test_datagen = tf.keras.preprocessing.image.ImageDataGenerator()
        n = str.replace(str(nl), '.', '_')
        test_generator = test_datagen.flow_from_directory(
            'data/RSNA_BME548-png/test_noisy_'+n,
            target_size=(224, 224),
            batch_size=4,
            class_mode='binary'
        )

        test_gens[nl] = test_generator

    return test_gens

In [30]:
noise_data_gens = get_test_data()

Found 800 images belonging to 2 classes.
Found 800 images belonging to 2 classes.


Found 800 images belonging to 2 classes.
Found 800 images belonging to 2 classes.
Found 800 images belonging to 2 classes.
Found 800 images belonging to 2 classes.
Found 800 images belonging to 2 classes.
Found 800 images belonging to 2 classes.
Found 800 images belonging to 2 classes.
Found 800 images belonging to 2 classes.


In [ ]:
# Define the class names
class_names = ['Hemorrhage', 'No Hemorrhage']

accuracies = []
all_trues = []
all_preds = []

for noise_level in TEST_NOISE_LEVELS:
    y_pred = get_predictions(model, noise_data_gens[noise_level])
    labels = noise_data_gens[noise_level].labels
    accuracy = metrics.accuracy_score(labels, y_pred)
    print(f"Accuracy of model on noise level {noise_level} = {accuracy}")
    accuracies.append(accuracy)
    all_trues.extend(labels)
    all_preds.extend(y_pred)


In [ ]:
# Get classification report
class_report = get_classification_report(all_trues, all_preds)
# Convert noise level to a valid string for the filename
filename = f"report_{str(NOISE_LEVEL).replace('.', '_')}_level.txt"
# Save classification report
with open(f"data/outputs/classification_reports/{filename}", "w") as file:
    file.write(class_report)

In [ ]:
# show confusion matrix
confusion_matrix = metrics.confusion_matrix(all_trues, all_preds)
print(confusion_matrix)